<a href="https://colab.research.google.com/github/sridharsakkari/sree/blob/master/Copy_of_Image_captioning_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Develop a model to determine the image captioning by using pretrained VGG16 model and CNN-RNN Architecture.

In [1]:
import zipfile
from zipfile import ZipFile

zipped_imgfile = ZipFile('/content/drive/My Drive/Image_captioning/Flickr8k_Dataset.zip','r')
zipped_imgfile.extractall()

zipped_captfile = ZipFile('/content/drive/My Drive/Image_captioning/Flickr8k_text.zip','r')
zipped_captfile.extractall()

In [2]:
import string
import numpy as np
from PIL import Image

import os
from pickle import dump,load

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout


# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


From the text folder we have imported, all image captions are stored in the file Flickr8k.token.txt. 


In [3]:
# loading a file 

def load_file(filename):

  file = open(filename,'r')
  text = file.read()
  file.close()

  return text


# get all images with their captions 

def image_captions(filename):

  file = load_file(filename)
  captions = file.split('\n') # as every caption is in new line

  descriptions = []
  images = []

  for caption in captions[:-1]: #last value in captions file is ''
    
    img,capt = caption.split('\t')

    descriptions.append(capt)
    images.append(img[:-2])

  return images,descriptions

In [4]:
text = load_file('Flickr8k.token.txt')
images,descriptions = image_captions('Flickr8k.token.txt')

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [134]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import string

In [347]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.utils import plot_model

In [51]:
def feature_extraction_from_images(Image_dir):

  model = VGG16()

  model.layers.pop()

  model = Model(inputs = model.inputs,outputs = model.layers[-1].output)

  print(model.summary())

  # extract features from each image

  features = dict()
  Image_path = os.listdir(Image_dir) 
  for name in Image_path:

    image = load_img(Image_dir+name,target_size=(224,224))

    image = img_to_array(image)

    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))

    image = preprocess_input(image)

    feature = model.predict(image,verbose=0)

    image_id = name.split('.')[0]

    features[image_id] = feature

  return features


   



  

In [32]:
Image_dir = ('/content/Flicker8k_Dataset/')


In [52]:
features = feature_extraction_from_images(Image_dir)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [53]:
dump(features,open('features.pkl', 'wb'))  #dump is used to create pickle file and save features into it


In [132]:
import string

At first, the text data is prepared by loading the files from the root directory and perform cleaning of the vocabulary data.

step 1 : Load the raw file consists of tokens by using the load_doc function. 

step 2 : The loaded document is passed through load_descriptions function where tha data is split into image identifier and description.  Every identifier has multiple probable captions. 

step 3 : Now the descriptions are cleaned word to word by using some tokenization techniques.. by function clean_descriptions

step 4 : convert the cleaned list into a vocabulary of words by to_vocabulary function.

step 5 : save the file to a descriptions.txt file as a vocab dictionary


step 6 : Load the traindataset, valid dataset and test dataset




In [290]:
text = load_doc('Flickr8k.token.txt')
text[:166]

'1000268201_693b08cb0e.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .\n1000268201_693b08cb0e.jpg#1\tA girl going into a wooden building .'

In [262]:
mapping = load_descriptions(text)

In [279]:
list(mapping.items())[1]

('1001773457_577c3a7d70',
 ['A black dog and a spotted dog are fighting',
  'A black dog and a tri-colored dog playing with each other on the road .',
  'A black dog and a white dog with brown spots are staring at each other in the street .',
  'Two dogs of different breeds looking at each other on the road .',
  'Two dogs on pavement moving toward each other .'])

In [318]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text



# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):   #split the data as per linewise i.e. '\n'
		
		tokens = line.split()         # split line by white space
		if len(line) < 2:
			continue
		image_id, image_desc = tokens[0], tokens[1:]    # take the first token as the image id, the rest as the description

		image_id = image_id.split('.')[0]       # remove filename from image id i.e. file format is removed from the identifier

		image_desc = ' '.join(image_desc)       # convert description tokens back to string

		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
  

	return mapping



def clean_descriptions(descriptions):
	
	table = str.maketrans('', '', string.punctuation)      # prepare translation table for removing punctuation

	for key, desc_list in descriptions.items():            # descriptions is a dictionary with image identifiers mapping with captions..
                                                         # key : identifiers, desc_list : captions ....(1,2,...)
   
		for i in range(len(desc_list)):                      # len of captions... i.e. numnber of captions in the list        
    
			desc = desc_list[i]                                # extract one element

			# tokenize

			desc = desc.split()                                # split the caption according to whitespaces
			
			desc = [word.lower() for word in desc]             # convert to lower case
			
			desc = [w.translate(table) for w in desc]          # remove punctuation from each token
			
			desc = [word for word in desc if len(word)>1]      # remove hanging 's' and 'a'
			
			desc = [word for word in desc if word.isalpha()]   # remove tokens with numbers in them
			
			desc_list[i] =  ' '.join(desc)                     # store as string



# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings

	all_desc = set()
 
	for key in descriptions.keys():
   
		[all_desc.update(d.split()) for d in descriptions[key]]         
	return all_desc



# save descriptions to file, one per line
def save_descriptions(descriptions, filename):             # save the vocab list to a file descriptions.txt
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

filename = 'Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, 'descriptions.txt')

Loaded: 8092 
Vocabulary Size: 8763


In [319]:
# loading data train,valid,test data 


def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

In [320]:
#load photos from train /test dataset and below function will load clean descriptions

def load_clean_descriptions(filename, dataset):

  doc = load_doc(filename)

  descriptions = {}

  for line in doc.split('\n'):

    if len(line)>2:
    
     tokens = line.split()
     
     image_id,image_desc = tokens[0], tokens[1:]

    if image_id in dataset:

      

      if image_id not in descriptions:

        descriptions[image_id] = list()

      desc = 'startseq ' + ' '.join(image_desc) + ' endseq'

      descriptions[image_id].append(desc)

  return descriptions

In [321]:
dataset = load_set('Flickr_8k.trainImages.txt')
descriptions = load_clean_descriptions('descriptions.txt',dataset)

In [322]:
def photo_features(filename,dataset):
  all_photo_features = load(open(filename,'rb'))
  features = {k:all_photo_features[k] for k in dataset}

  return features

In [323]:
photofeatures = photo_features('features.pkl',dataset)

Before feeding the processed data to the model, the data shall be encoded to numbers.
Create a consistent mapping from words to unique integer values

In keras, we have a tokenizer class to perform this function

In [324]:
def to_lines(descriptions):      # convert a cleaned dictionary of tokens to list

  all_desc = list()

  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]

  return all_desc


In [329]:
# fit a tokenizer 

def create_tokenizer(descriptions):
  lines_ = to_lines(descriptions)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines_)

  return tokenizer

In [330]:
tokenizer = create_tokenizer(descriptions)

In [335]:
vocab_size = len(tokenizer.word_index) +1

In [342]:
def max_length(descriptions):

  lines_ = to_lines(descriptions)

  return max(len(d.split()) for d in lines_)

In [337]:
def create_sequences(tokenizer,max_length,descriptions,photos,vocab_size):

  X1,X2,y = list(),list(),list()

  for key,desc_list in descriptions.items():

    for desc in desc_list:

      seq = tokenizer.texts_to_sequences([desc])[0]

      for i in range(1,len(seq)):

        in_seq,out_seq = seq[:i],seq[i]

        in_seq = pad_sequences([in_seq],maxlen =max_length)[0]

        out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]

        X1.append(photos[key][0])
        X2.append(in_seq)
        y.append(out_seq)

  return array(X1),array(X2),array(y)


Step 1 : Photo Feature Extractor : images are processed through pretrained vgg16 model without output layer .

Step 2 : Word embedding layer followed by LSTM layer  (sequence processor)

Step 3 : Decoder: both a feature extractor and sequence processor output a fixed length vector. 
 

In [343]:
max_length(descriptions)

34

The sequence processor produces a word embedding layer of length 34 words.
The photo extractor produces an output of size 4096.
this is processed through a dense layer to produce a 256 sized vector



In [348]:
def model2(vocab_size,max_length):

  # feature extractor
  inputs1 = Input(shape = (4096,))
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256,activation='relu')(fe1)

  # sequence processor 

  inputs2 = Input(shape=(max_length,))
  se1 = Embedding(vocab_size,256,mask_zero =True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)


  # decoder model

  decoder1 = add([fe2,se3])
  decoder2 = Dense(256,activation='relu')(decoder1)
  outputs = Dense(vocab_size,activation='softmax')(decoder2)

  model = Model(inputs = [inputs1,inputs2],outputs = outputs)
  model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')

  print(model.summary())

  plot_model(model, to_file='model.png', show_shapes=True)
	
  return model




In [350]:
model2(vocab_size,34)

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 34)           0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 34, 256)      1940224     input_12[0][0]                   
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 4096)         0           input_11[0][0]                   
____________________________________________________________________________________________